# GAMA-15 master catalogue
## Preparation of KIDS/VST data

Kilo Degree Survey/VLT Survey Telescope catalogue: the catalogue comes from `dmu0_KIDS`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The aperture corrected aperture magnitude in each band (10 pixels = 2")
- The Petrosian magnitude to be used as total magnitude (no “auto” magnitude is provided).

We take 2014 as the observation year from a typical image header.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux, flux_to_mag

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL  = "kids_ra"
DEC_COL = "kids_dec"

## I - Column selection

In [ ]:
imported_columns = OrderedDict({
        'ID': "kids_id",
        'RAJ2000': "kids_ra",
        'DECJ2000': "kids_dec",
        'CLASS_STAR':  "kids_stellarity",
        'MAG_AUTO_U': "m_kids_u", 
        'MAGERR_AUTO_U': "merr_kids_u", 
        'MAG_AUTO_G': "m_kids_g", 
        'MAGERR_AUTO_G': "merr_kids_g", 
        'MAG_AUTO_R': "m_kids_r", 
        'MAGERR_AUTO_R': "merr_kids_r", 
        'MAG_AUTO_I': "m_kids_i", 
        'MAGERR_AUTO_I': "merr_kids_i", 
        'FLUX_APERCOR_10_U': "f_ap_kids_u",
        'FLUXERR_APERCOR_10_U': "ferr_ap_kids_u",
        'FLUX_APERCOR_10_G': "f_ap_kids_g",
        'FLUXERR_APERCOR_10_G': "ferr_ap_kids_g",
        'FLUX_APERCOR_10_R': "f_ap_kids_r",
        'FLUXERR_APERCOR_10_R': "ferr_ap_kids_r",
        'FLUX_APERCOR_10_I': "f_ap_kids_i",
        'FLUXERR_APERCOR_10_I': "ferr_ap_kids_i"

    })


catalogue = Table.read("../../dmu0/dmu0_KIDS/data/KIDS-DR3_GAMA-15.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2014 #A range of observation dates from 2011 to 2015.

# Clean table metadata
catalogue.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])

        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
    if col.startswith('f_'):
        
        errcol = "ferr{}".format(col[1:])
        
        #Convert fluxes in maggies to uJy
        catalogue[col] *= 3631. * 1.e6
        catalogue[col].unit = 'uJy'
        catalogue[errcol] *= 3631. * 1.e6
        catalogue[errcol].unit = 'uJy'

        mag, mag_error = flux_to_mag(np.array(catalogue[col]) * 1.e-6, 
                                     np.array(catalogue[errcol]) * 1.e-6)
        
        # Magnitudes are added
        catalogue.add_column(Column(mag, name="m{}".format(col[1:])))
        catalogue.add_column(Column(mag_error, name="m{}".format(errcol[1:])))
        

In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_kids_u', 
             'merr_ap_kids_g', 
             'merr_ap_kids_r', 
             'merr_ap_kids_i']
FLAG_NAME = 'kids_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_GAMA-15.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "kids_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/KIDS.fits".format(OUT_DIR), overwrite=True)